In [1]:
import pandas as pd
import numpy as np

fp = "data/filter/en/filtered_data1703.csv"

df = pd.read_csv(fp)
df = df.loc[df["created_at"].str.contains("2021")]

In [7]:
# not used anymore, takes too much time
#all_tweets = pd.read_csv("data/TweetSearch.csv", low_memory=False)

In [2]:
# change datetimes to correct format so it doesn't need to be done later in many different places

df["created_at"] = pd.to_datetime(df["created_at"], format="%Y-%m-%dT%H:%M:%S%z")
df["user.created_at"] = pd.to_datetime(df["user.created_at"], format="%Y-%m-%dT%H:%M:%S%z")

#all_tweets["created_at"] = pd.to_datetime(all_tweets["created_at"], format="%Y-%m-%dT%H:%M:%S%z")
#all_tweets["user.created_at"] = pd.to_datetime(all_tweets["user.created_at"], format="%Y-%m-%dT%H:%M:%S%z")
#df

## Explore/filter

In [10]:
# probably need to remove #bluenile, could keep tweets mentioning also renaissance dam

#df.loc[df["text"].str.lower().str.contains(r'^(?=.*bluenile)(?=.*ethiopia)')]["text"].to_list()

In [6]:
lst = ["#GERD", "#ItsMyDam", "#ItsOurDam", "#ItsHerDam", "#FillTheDam", "#FillTheGERD", "#FinishTheGERD", "#AbiyIsTheMan"]
#["millennium dam", "renaissance dam", "ethiopian dam", "hydropower", "hydro-power", 
 #      "hydro power", "water security", "watersecurity", "hydrodiplomacy", 
  #     "nile basin initiative", "cooperative framework"]

tweets = df.loc[df["text"].str.lower().str.contains("|".join([x.lower() for x in lst]))].reset_index(drop=True).copy()#["text"].to_list()
print(len(tweets), "tweets selected")

87286 tweets selected


In [8]:
n = 0

for i in tweets.columns:
    print(n, ":", i)
    n += 1

0 : Unnamed: 0
1 : created_at
2 : id
3 : author_id
4 : conversation_id
5 : reply_settings
6 : text
7 : lang
8 : public_metrics.retweet_count
9 : public_metrics.reply_count
10 : public_metrics.like_count
11 : public_metrics.quote_count
12 : possibly_sensitive
13 : referenced_tweets.id
14 : referenced_tweets.type
15 : in_reply_to_user_id
16 : user.public_metrics.followers_count
17 : user.public_metrics.following_count
18 : user.public_metrics.tweet_count
19 : user.public_metrics.listed_count
20 : user.url
21 : user.id
22 : user.username
23 : user.profile_image_url
24 : user.created_at
25 : user.location
26 : user.description
27 : day
28 : count


## Mentions

In [9]:
tweets["mentions"] = ""

# fill column mentions

for i in range(len(tweets)):
    temp = [t for t in tweets["text"][i].split() if t.startswith("@")]
    for j in range(len(temp)):
        temp[j] = temp[j].strip(",.!?:;")
    if temp:
        tweets.iat[i, 29] = temp
        
tweets.loc[tweets["mentions"].isnull() == False].head()

,Unnamed: 0,created_at,id,author_id,conversation_id,reply_settings,text,lang,public_metrics.retweet_count,public_metrics.reply_count,...,user.url,user.id,user.username,user.profile_image_url,user.created_at,user.location,user.description,day,count,mentions
0,51854,2021-01-01 21:12:13+00:00,1345115637979426819,1301826968036560896,1344676938284195842,everyone,@iyoba4u Why not withdraw Eritrean forces from...,en,0,0,...,NaN,1301826968036560896,2020Tgi20202,https://pbs.twimg.com/profile_images/135027117...,2020-09-04 10:19:19+00:00,NaN,NaN,2021-01-01,24,[@iyoba4u]
1,51856,2021-01-01 21:11:37+00:00,1345115487269761026,32277173,1345091357702987777,everyone,@BashirHashiysf It’s time #Egypt gets involved...,en,3,0,...,NaN,32277173,alulagebru,https://pbs.twimg.com/profile_images/158215310...,2009-04-17 03:03:58+00:00,Right Here,Dedicated to the people of Tigray• Tigray is a...,2021-01-01,24,[@BashirHashiysf]
2,51857,2021-01-01 00:10:23+00:00,1344798086649044994,32277173,1344684285090000899,everyone,@DrMehari Congratulations #Sudan! Everyone saw...,en,0,0,...,NaN,32277173,alulagebru,https://pbs.twimg.com/profile_images/158215310...,2009-04-17 03:03:58+00:00,Right Here,Dedicated to the people of Tigray• Tigray is a...,2021-01-01,24,[@DrMehari]
3,51860,2021-01-01 15:53:24+00:00,1345035405272481793,1278701315347099649,1345035405272481793,everyone,Egyptian authorities silence anyone who challe...,en,0,1,...,NaN,1278701315347099649,JES_BH,https://pbs.twimg.com/profile_images/136108165...,2020-07-02 14:45:32+00:00,NaN,Unapologetically Anti-imperialist,2021-01-01,24,
4,51861,2021-01-01 15:32:52+00:00,1345030235981541376,1227514178493145088,1345030235981541376,everyone,#Ethiopia will achieve another victory in 2021...,en,0,0,...,NaN,1227514178493145088,zerihunmegersaj,https://pbs.twimg.com/profile_images/159035406...,2020-02-12 08:46:18+00:00,Ethiopia,Serving #Ethiopia🇪🇹 is a great Honor!!!\r\nPer...,2021-01-01,24,


In [10]:
row = 140

print(tweets.iat[row, 29])
print(tweets.iat[row, 6])


Yes. #GERD saved #Sudan and will continue to save from devastating floods. #GERD has immense benefits to our #Sudanese brothers and sisters.

Egypt doesn't care if Khartoum gets destroyed by devastating floods. Egypt opposes to #GERD because Egypt is the enemy of Sudanese people. https://t.co/IdYXZxyGMj


In [11]:
mentions = tweets.loc[tweets["mentions"] != ""].reset_index(drop=True).copy()

mentions = mentions.explode("mentions").reset_index(drop=True)
mentions["mentions"] = mentions["mentions"].str.lower().str.strip("@")

ids = pd.read_csv("data/other/username_id_all.csv").rename(columns={"usernames_target":"mentions", "user.id":"in_reply_to_user_id"})
edges_mention = mentions.drop(columns=["in_reply_to_user_id"]).merge(ids, how="left", on="mentions").copy()
edges_mention = edges_mention.loc[edges_mention["in_reply_to_user_id"].isnull() == False].reset_index(drop=True)
edges_mention["in_reply_to_user_id"] = edges_mention["in_reply_to_user_id"].astype(np.int64)
edges_mention["connection"] = "mention"
edges_mention

,Unnamed: 0,created_at,id,author_id,conversation_id,reply_settings,text,lang,public_metrics.retweet_count,public_metrics.reply_count,...,user.username,user.profile_image_url,user.created_at,user.location,user.description,day,count,mentions,in_reply_to_user_id,connection
0,51854,2021-01-01 21:12:13+00:00,1345115637979426819,1301826968036560896,1344676938284195842,everyone,@iyoba4u Why not withdraw Eritrean forces from...,en,0,0,...,2020Tgi20202,https://pbs.twimg.com/profile_images/135027117...,2020-09-04 10:19:19+00:00,NaN,NaN,2021-01-01,24,iyoba4u,3740334387,mention
1,51856,2021-01-01 21:11:37+00:00,1345115487269761026,32277173,1345091357702987777,everyone,@BashirHashiysf It’s time #Egypt gets involved...,en,3,0,...,alulagebru,https://pbs.twimg.com/profile_images/158215310...,2009-04-17 03:03:58+00:00,Right Here,Dedicated to the people of Tigray• Tigray is a...,2021-01-01,24,bashirhashiysf,636089833,mention
2,51857,2021-01-01 00:10:23+00:00,1344798086649044994,32277173,1344684285090000899,everyone,@DrMehari Congratulations #Sudan! Everyone saw...,en,0,0,...,alulagebru,https://pbs.twimg.com/profile_images/158215310...,2009-04-17 03:03:58+00:00,Right Here,Dedicated to the people of Tigray• Tigray is a...,2021-01-01,24,drmehari,357484690,mention
3,51865,2021-01-01 11:27:45+00:00,1344968549056524288,247298738,1344968549056524288,everyone,Congrats @_AfricanUnion on launching #AfCFTA. ...,en,0,0,...,views_of_ado,https://pbs.twimg.com/profile_images/125709925...,2011-02-04 14:22:37+00:00,Ethiopia,"Founder #FoundersFuel, @LigabaTaxi, @DigitalPa...",2021-01-01,24,_africanunion,145935897,mention
4,51865,2021-01-01 11:27:45+00:00,1344968549056524288,247298738,1344968549056524288,everyone,Congrats @_AfricanUnion on launching #AfCFTA. ...,en,0,0,...,views_of_ado,https://pbs.twimg.com/profile_images/125709925...,2011-02-04 14:22:37+00:00,Ethiopia,"Founder #FoundersFuel, @LigabaTaxi, @DigitalPa...",2021-01-01,24,hmdessalegn,1226783393888768000,mention
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285046,155487,2021-12-31 04:29:34+00:00,1476772493125767171,127362490,1476772493125767171,everyone,Ethiopian Dam &amp; Anti-Ethiopia Global Consp...,en,1,0,...,dereje_1,https://pbs.twimg.com/profile_images/104360380...,2010-03-28 23:52:36+00:00,United States,Ethinicity= Mixed= Ethiopian,2021-12-31,29,cyberarmyeth,753885,mention
285047,155487,2021-12-31 04:29:34+00:00,1476772493125767171,127362490,1476772493125767171,everyone,Ethiopian Dam &amp; Anti-Ethiopia Global Consp...,en,1,0,...,dereje_1,https://pbs.twimg.com/profile_images/104360380...,2010-03-28 23:52:36+00:00,United States,Ethinicity= Mixed= Ethiopian,2021-12-31,29,fdreservice,1451174958021873664,mention
285048,155487,2021-12-31 04:29:34+00:00,1476772493125767171,127362490,1476772493125767171,everyone,Ethiopian Dam &amp; Anti-Ethiopia Global Consp...,en,1,0,...,dereje_1,https://pbs.twimg.com/profile_images/104360380...,2010-03-28 23:52:36+00:00,United States,Ethinicity= Mixed= Ethiopian,2021-12-31,29,billeneseyoum,707189905,mention
285049,155487,2021-12-31 04:29:34+00:00,1476772493125767171,127362490,1476772493125767171,everyone,Ethiopian Dam &amp; Anti-Ethiopia Global Consp...,en,1,0,...,dereje_1,https://pbs.twimg.com/profile_images/104360380...,2010-03-28 23:52:36+00:00,United States,Ethinicity= Mixed= Ethiopian,2021-12-31,29,neaminzeleke,4077439067,mention


## Replys

In [70]:
replys = tweets.loc[tweets["referenced_tweets.id"].isnull() == False].copy()

In [71]:
len(replys)

19746

In [72]:
replys["referenced_tweets.id"] = replys.loc[:, "referenced_tweets.id"].values.astype(str)

replys["referenced_tweets.id"] = replys.loc[:, "referenced_tweets.id"].str.split(";")
replys["referenced_tweets.type"] = replys.loc[:, "referenced_tweets.type"].str.split(";")

replys = replys.explode(["referenced_tweets.id", "referenced_tweets.type"]).reset_index(drop=True)

replys["referenced_tweets.id"] = pd.to_numeric(replys["referenced_tweets.id"]).astype(np.int64)

# create dataframe with user.id and (tweet) id; merge on referenced_tweets.id with replys to create a new column with target user id; fix finalizing part

tweet_user_id = pd.DataFrame({"in_reply_to_user_id" : df["user.id"], "referenced_tweets.id" : df["id"]}).copy()

replys = replys.drop(columns="in_reply_to_user_id").merge(tweet_user_id, on="referenced_tweets.id", how="left")
replys = replys.loc[replys["in_reply_to_user_id"].isnull() == False]
replys["in_reply_to_user_id"] = pd.to_numeric(replys["in_reply_to_user_id"]).astype(np.int64)
replys

,Unnamed: 0,created_at,id,author_id,conversation_id,reply_settings,text,lang,public_metrics.retweet_count,public_metrics.reply_count,...,user.id,user.username,user.profile_image_url,user.created_at,user.location,user.description,day,count,mentions,in_reply_to_user_id
16,51891,2021-01-02 18:54:59+00:00,1345443490164047872,58726381,1345443485554503680,everyone,When #alsisi came to power the #USA did not wa...,en,0,1,...,58726381,kassaahunt,https://pbs.twimg.com/profile_images/146157640...,2009-07-21 06:39:02+00:00,ህይወት በወሰደችኝ ቦታ የምገኝ,#አፍሪቃዊ #ኢትዮጵያዊ #አዲስአበቤ #ነፃነት #ሰብዓዊነት,2021-01-02,40,,58726381
22,51916,2021-01-03 21:25:17+00:00,1345843699817861122,1265237229393362945,1345843699817861122,everyone,#GERD talks failed again.\r\n#Egypt says no de...,en,0,0,...,1265237229393362945,nomorewarTIG,https://pbs.twimg.com/profile_images/135602530...,2020-05-26 11:04:12+00:00,NaN,Art & Peace •\r\n\r\nCurrently stateless • \r...,2021-01-03,121,,1221534859
23,51918,2021-01-03 22:53:20+00:00,1345865859542167552,246020109,1345865859542167552,everyone,#EthiopiaRising #GERD No matter what obstacles...,en,0,0,...,246020109,meru_maj,https://pbs.twimg.com/profile_images/128089541...,2011-02-01 23:33:18+00:00,NaN,🇪🇹🇪🇷🇨🇦,2021-01-03,121,,1221534859
24,51921,2021-01-03 17:46:02+00:00,1345788524310781952,32277173,1345766618882854915,everyone,@seleshi_b_a “Egypt categorically rejected the...,en,2,15,...,32277173,alulagebru,https://pbs.twimg.com/profile_images/158215310...,2009-04-17 03:03:58+00:00,Right Here,Dedicated to the people of Tigray• Tigray is a...,2021-01-03,121,[@seleshi_b_a],4160135608
27,51928,2021-01-03 12:48:34+00:00,1345713666902126595,3740334387,1345713666902126595,everyone,"Unless #Ethiopia'n gov't have good reason, the...",en,0,1,...,3740334387,iyoba4u,https://pbs.twimg.com/profile_images/151487217...,2015-09-22 16:12:11+00:00, 🐦,"'We Have To Live The Dream, Not Just Dream' - ...",2021-01-03,121,,4160135608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19938,155331,2021-12-26 10:27:51+00:00,1475050720500097025,963847516285489152,1474954202061410304,everyone,"@AntenehMelesse Well done! Viva ##Ethiopia, Vi...",en,0,0,...,963847516285489152,dagu_sa,https://pbs.twimg.com/profile_images/161160676...,2018-02-14 18:48:50+00:00,NaN,NaN,2021-12-26,46,[@AntenehMelesse],1667176201
19943,155342,2021-12-27 14:11:01+00:00,1475469268288425985,2538589270,1475059875428417538,everyone,"Even if the world is against you, if you clear...",en,5,0,...,2538589270,FuadMustefaAli,https://pbs.twimg.com/profile_images/161790546...,2014-05-10 07:36:59+00:00,Ethiopia,⚖|Muslim|🇪🇹n|Loving Husband|Proud Father|Educa...,2021-12-27,29,"[@iyoba4u, @NeaminZeleke, @seleshi_b_a]",2538589270
19955,155373,2021-12-28 17:52:51+00:00,1475887484227620870,1237412915331788801,1475790086247694340,everyone,"@NginyaDiyu Yeap, #NoMoreDarkness\r\n🇪🇹prevail...",en,0,0,...,1237412915331788801,TsedaleAbebe,https://pbs.twimg.com/profile_images/162186869...,2020-03-10 16:22:00+00:00,NaN,NaN,2021-12-28,40,[@NginyaDiyu],4587564076
19965,155408,2021-12-29 20:52:42+00:00,1476295131645026307,1374346395474354176,1476245937819430921,everyone,@xurafel @RAbdiAnalyst The only positive thing...,en,0,2,...,1374346395474354176,Sara18785983,https://pbs.twimg.com/profile_images/160307907...,2021-03-23 13:05:01+00:00,NaN,Views are my own,2021-12-29,27,"[@xurafel, @RAbdiAnalyst]",777884179


In [73]:
replys = replys.reset_index(drop=True)
replys["connection"] = "reply"
replys

,Unnamed: 0,created_at,id,author_id,conversation_id,reply_settings,text,lang,public_metrics.retweet_count,public_metrics.reply_count,...,user.username,user.profile_image_url,user.created_at,user.location,user.description,day,count,mentions,in_reply_to_user_id,connection
0,51891,2021-01-02 18:54:59+00:00,1345443490164047872,58726381,1345443485554503680,everyone,When #alsisi came to power the #USA did not wa...,en,0,1,...,kassaahunt,https://pbs.twimg.com/profile_images/146157640...,2009-07-21 06:39:02+00:00,ህይወት በወሰደችኝ ቦታ የምገኝ,#አፍሪቃዊ #ኢትዮጵያዊ #አዲስአበቤ #ነፃነት #ሰብዓዊነት,2021-01-02,40,,58726381,reply
1,51916,2021-01-03 21:25:17+00:00,1345843699817861122,1265237229393362945,1345843699817861122,everyone,#GERD talks failed again.\r\n#Egypt says no de...,en,0,0,...,nomorewarTIG,https://pbs.twimg.com/profile_images/135602530...,2020-05-26 11:04:12+00:00,NaN,Art & Peace •\r\n\r\nCurrently stateless • \r...,2021-01-03,121,,1221534859,reply
2,51918,2021-01-03 22:53:20+00:00,1345865859542167552,246020109,1345865859542167552,everyone,#EthiopiaRising #GERD No matter what obstacles...,en,0,0,...,meru_maj,https://pbs.twimg.com/profile_images/128089541...,2011-02-01 23:33:18+00:00,NaN,🇪🇹🇪🇷🇨🇦,2021-01-03,121,,1221534859,reply
3,51921,2021-01-03 17:46:02+00:00,1345788524310781952,32277173,1345766618882854915,everyone,@seleshi_b_a “Egypt categorically rejected the...,en,2,15,...,alulagebru,https://pbs.twimg.com/profile_images/158215310...,2009-04-17 03:03:58+00:00,Right Here,Dedicated to the people of Tigray• Tigray is a...,2021-01-03,121,[@seleshi_b_a],4160135608,reply
4,51928,2021-01-03 12:48:34+00:00,1345713666902126595,3740334387,1345713666902126595,everyone,"Unless #Ethiopia'n gov't have good reason, the...",en,0,1,...,iyoba4u,https://pbs.twimg.com/profile_images/151487217...,2015-09-22 16:12:11+00:00, 🐦,"'We Have To Live The Dream, Not Just Dream' - ...",2021-01-03,121,,4160135608,reply
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5984,155331,2021-12-26 10:27:51+00:00,1475050720500097025,963847516285489152,1474954202061410304,everyone,"@AntenehMelesse Well done! Viva ##Ethiopia, Vi...",en,0,0,...,dagu_sa,https://pbs.twimg.com/profile_images/161160676...,2018-02-14 18:48:50+00:00,NaN,NaN,2021-12-26,46,[@AntenehMelesse],1667176201,reply
5985,155342,2021-12-27 14:11:01+00:00,1475469268288425985,2538589270,1475059875428417538,everyone,"Even if the world is against you, if you clear...",en,5,0,...,FuadMustefaAli,https://pbs.twimg.com/profile_images/161790546...,2014-05-10 07:36:59+00:00,Ethiopia,⚖|Muslim|🇪🇹n|Loving Husband|Proud Father|Educa...,2021-12-27,29,"[@iyoba4u, @NeaminZeleke, @seleshi_b_a]",2538589270,reply
5986,155373,2021-12-28 17:52:51+00:00,1475887484227620870,1237412915331788801,1475790086247694340,everyone,"@NginyaDiyu Yeap, #NoMoreDarkness\r\n🇪🇹prevail...",en,0,0,...,TsedaleAbebe,https://pbs.twimg.com/profile_images/162186869...,2020-03-10 16:22:00+00:00,NaN,NaN,2021-12-28,40,[@NginyaDiyu],4587564076,reply
5987,155408,2021-12-29 20:52:42+00:00,1476295131645026307,1374346395474354176,1476245937819430921,everyone,@xurafel @RAbdiAnalyst The only positive thing...,en,0,2,...,Sara18785983,https://pbs.twimg.com/profile_images/160307907...,2021-03-23 13:05:01+00:00,NaN,Views are my own,2021-12-29,27,"[@xurafel, @RAbdiAnalyst]",777884179,reply


## Used hashtags

In [74]:
tweets["hashtags"] = ""

# fill column mentions

for i in range(len(tweets)):
    temp = [t for t in tweets["text"][i].split() if t.startswith("#")]
    for j in range(len(temp)):
        temp[j] = temp[j].strip(",.!?:;")
    if temp:
        tweets.iat[i, 30] = temp
        
tweets.loc[tweets["hashtags"].isnull() == False].head()

,Unnamed: 0,created_at,id,author_id,conversation_id,reply_settings,text,lang,public_metrics.retweet_count,public_metrics.reply_count,...,user.id,user.username,user.profile_image_url,user.created_at,user.location,user.description,day,count,mentions,hashtags
0,51854,2021-01-01 21:12:13+00:00,1345115637979426819,1301826968036560896,1344676938284195842,everyone,@iyoba4u Why not withdraw Eritrean forces from...,en,0,0,...,1301826968036560896,2020Tgi20202,https://pbs.twimg.com/profile_images/135027117...,2020-09-04 10:19:19+00:00,NaN,NaN,2021-01-01,24,[@iyoba4u],"[#Tigray, #Ethiopia, #GERD, #Ethiopia]"
1,51856,2021-01-01 21:11:37+00:00,1345115487269761026,32277173,1345091357702987777,everyone,@BashirHashiysf It’s time #Egypt gets involved...,en,3,0,...,32277173,alulagebru,https://pbs.twimg.com/profile_images/158215310...,2009-04-17 03:03:58+00:00,Right Here,Dedicated to the people of Tigray• Tigray is a...,2021-01-01,24,[@BashirHashiysf],"[#Egypt, #GERD]"
2,51857,2021-01-01 00:10:23+00:00,1344798086649044994,32277173,1344684285090000899,everyone,@DrMehari Congratulations #Sudan! Everyone saw...,en,0,0,...,32277173,alulagebru,https://pbs.twimg.com/profile_images/158215310...,2009-04-17 03:03:58+00:00,Right Here,Dedicated to the people of Tigray• Tigray is a...,2021-01-01,24,[@DrMehari],"[#Sudan, #Sudan, #GERD.#SanctionEthiopia]"
3,51858,2021-01-01 18:56:47+00:00,1345081554532446208,1308594900,1345081554532446208,everyone,Meet #Eritrea of #BlueNile diverse pop and lan...,en,0,0,...,1308594900,nevuab,https://pbs.twimg.com/profile_images/118706742...,2013-03-27 21:00:20+00:00,"New York, NY",Coptic+Catholic Music Trained & Former Marcher...,2021-01-01,24,,"[#Eritrea, #BlueNile]"
4,51860,2021-01-01 15:53:24+00:00,1345035405272481793,1278701315347099649,1345035405272481793,everyone,Egyptian authorities silence anyone who challe...,en,0,1,...,1278701315347099649,JES_BH,https://pbs.twimg.com/profile_images/136108165...,2020-07-02 14:45:32+00:00,NaN,Unapologetically Anti-imperialist,2021-01-01,24,,"[#Ethiopians, #GERD, #Ethiopia, #GuilioRegeni]"


In [81]:
hashtags = tweets.loc[tweets["hashtags"] != ""].reset_index(drop=True).copy()

hashtags = hashtags.explode("hashtags").reset_index(drop=True)
hashtags["hashtags"] = hashtags["hashtags"].str.lower().str.strip("#")

hashtags["hashtags"]

ht = pd.read_csv("data/other/nodes_hashtags.csv").rename(columns={"Label":"hashtags", "id":"in_reply_to_user_id"})
ht["hashtags"] = ht["hashtags"].str.lower().str.strip("#")

edges_ht = hashtags.drop(columns=["in_reply_to_user_id"]).merge(ht.drop(columns=["nodetype"]), how="left", on="hashtags").copy()
edges_ht = edges_ht.loc[edges_ht["in_reply_to_user_id"].isnull() == False].reset_index(drop=True)
edges_ht["in_reply_to_user_id"] = edges_ht["in_reply_to_user_id"].astype(np.int64)
edges_ht["connection"] = "hashtag"
edges_ht

,Unnamed: 0,created_at,id,author_id,conversation_id,reply_settings,text,lang,public_metrics.retweet_count,public_metrics.reply_count,...,user.profile_image_url,user.created_at,user.location,user.description,day,count,mentions,hashtags,in_reply_to_user_id,connection
0,51854,2021-01-01 21:12:13+00:00,1345115637979426819,1301826968036560896,1344676938284195842,everyone,@iyoba4u Why not withdraw Eritrean forces from...,en,0,0,...,https://pbs.twimg.com/profile_images/135027117...,2020-09-04 10:19:19+00:00,NaN,NaN,2021-01-01,24,[@iyoba4u],tigray,42,hashtag
1,51854,2021-01-01 21:12:13+00:00,1345115637979426819,1301826968036560896,1344676938284195842,everyone,@iyoba4u Why not withdraw Eritrean forces from...,en,0,0,...,https://pbs.twimg.com/profile_images/135027117...,2020-09-04 10:19:19+00:00,NaN,NaN,2021-01-01,24,[@iyoba4u],ethiopia,4,hashtag
2,51854,2021-01-01 21:12:13+00:00,1345115637979426819,1301826968036560896,1344676938284195842,everyone,@iyoba4u Why not withdraw Eritrean forces from...,en,0,0,...,https://pbs.twimg.com/profile_images/135027117...,2020-09-04 10:19:19+00:00,NaN,NaN,2021-01-01,24,[@iyoba4u],gerd,12,hashtag
3,51854,2021-01-01 21:12:13+00:00,1345115637979426819,1301826968036560896,1344676938284195842,everyone,@iyoba4u Why not withdraw Eritrean forces from...,en,0,0,...,https://pbs.twimg.com/profile_images/135027117...,2020-09-04 10:19:19+00:00,NaN,NaN,2021-01-01,24,[@iyoba4u],ethiopia,4,hashtag
4,51856,2021-01-01 21:11:37+00:00,1345115487269761026,32277173,1345091357702987777,everyone,@BashirHashiysf It’s time #Egypt gets involved...,en,3,0,...,https://pbs.twimg.com/profile_images/158215310...,2009-04-17 03:03:58+00:00,Right Here,Dedicated to the people of Tigray• Tigray is a...,2021-01-01,24,[@BashirHashiysf],egypt,3,hashtag
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187723,155482,2021-12-31 09:45:19+00:00,1476851955087646720,192863506,1476851955087646720,everyone,A link to a new store page will be up soon. Th...,en,0,0,...,https://pbs.twimg.com/profile_images/140519441...,2010-09-20 11:01:08+00:00,London,"Critic, scribe, Ostalgist, @FCBayernEN and #Bo...",2021-12-31,29,,gerd,12,hashtag
187724,155485,2021-12-31 07:21:45+00:00,1476815826753839106,1380899221335867398,1476655235758276608,everyone,@mustafa__bag Is that reporting or Activism\r\...,en,1,0,...,https://pbs.twimg.com/profile_images/162397675...,2021-04-10 15:03:47+00:00,NaN,ሰው መሆን እንዴት መሆን ነው? \r\n\r\n777,2021-12-31,29,[@mustafa__bag],gerd,12,hashtag
187725,155487,2021-12-31 04:29:34+00:00,1476772493125767171,127362490,1476772493125767171,everyone,Ethiopian Dam &amp; Anti-Ethiopia Global Consp...,en,1,0,...,https://pbs.twimg.com/profile_images/104360380...,2010-03-28 23:52:36+00:00,United States,Ethinicity= Mixed= Ethiopian,2021-12-31,29,"[@Cyberarmyeth, @FdreService, @BilleneSeyoum, ...",tigray,42,hashtag
187726,155487,2021-12-31 04:29:34+00:00,1476772493125767171,127362490,1476772493125767171,everyone,Ethiopian Dam &amp; Anti-Ethiopia Global Consp...,en,1,0,...,https://pbs.twimg.com/profile_images/104360380...,2010-03-28 23:52:36+00:00,United States,Ethinicity= Mixed= Ethiopian,2021-12-31,29,"[@Cyberarmyeth, @FdreService, @BilleneSeyoum, ...",ethiopia,4,hashtag


## Nodes

In [12]:
sel = tweets.iloc[:, [21, 22, 24, 16, 17, 18, 19, 25, 26]]
nodes = sel.drop_duplicates(subset="user.id").reset_index(drop=True)
nodes["user.username"] = nodes["user.username"].str.lower()

In [13]:
# NOTE: requires "mentions" from section Edges: mentions

more_nodes = pd.DataFrame({"user.username" : mentions["mentions"]})
id_all = pd.read_csv("data/other/username_id_all.csv").rename(columns={"usernames_target":"user.username"})
more_nodes["user.id"] = np.nan

more_nodes = more_nodes.merge(id_all, how="left", on="user.username").drop(columns=["user.id_x"]).rename(columns={"user.id_y":"user.id"})

nodes_all = pd.concat([nodes, more_nodes]).drop_duplicates(subset="user.id").reset_index(drop=True)
nodes_all

,user.id,user.username,user.created_at,user.public_metrics.followers_count,user.public_metrics.following_count,user.public_metrics.tweet_count,user.public_metrics.listed_count,user.location,user.description
0,1.301827e+18,2020tgi20202,2020-09-04 10:19:19+00:00,1465.0,914.0,6659.0,3.0,NaN,NaN
1,3.227717e+07,alulagebru,2009-04-17 03:03:58+00:00,720.0,611.0,12619.0,5.0,Right Here,Dedicated to the people of Tigray• Tigray is a...
2,1.278701e+18,jes_bh,2020-07-02 14:45:32+00:00,319.0,476.0,2299.0,1.0,NaN,Unapologetically Anti-imperialist
3,1.227514e+18,zerihunmegersaj,2020-02-12 08:46:18+00:00,11576.0,907.0,7141.0,21.0,Ethiopia,Serving #Ethiopia🇪🇹 is a great Honor!!!\r\nPer...
4,1.166362e+18,acidrefluxfacts,2019-08-27 14:51:19+00:00,1387.0,4908.0,13496.0,1.0,NaN,Heartburn and Acid Reflux facts and treatment ...
...,...,...,...,...,...,...,...,...,...
18004,9.746740e+05,ethiopmedia,NaT,NaN,NaN,NaN,NaN,NaN,NaN
18005,3.060690e+05,_kheff,NaT,NaN,NaN,NaN,NaN,NaN,NaN
18006,2.110160e+05,realseyoum,NaT,NaN,NaN,NaN,NaN,NaN,NaN
18007,1.451175e+18,fdreservice,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
nodes_all#["user.username"].to_list()

,user.id,user.username,user.created_at,user.public_metrics.followers_count,user.public_metrics.following_count,user.public_metrics.tweet_count,user.public_metrics.listed_count,user.location,user.description
0,1.301827e+18,2020tgi20202,2020-09-04 10:19:19+00:00,1465.0,914.0,6659.0,3.0,NaN,NaN
1,3.227717e+07,alulagebru,2009-04-17 03:03:58+00:00,720.0,611.0,12619.0,5.0,Right Here,Dedicated to the people of Tigray• Tigray is a...
2,1.278701e+18,jes_bh,2020-07-02 14:45:32+00:00,319.0,476.0,2299.0,1.0,NaN,Unapologetically Anti-imperialist
3,1.227514e+18,zerihunmegersaj,2020-02-12 08:46:18+00:00,11576.0,907.0,7141.0,21.0,Ethiopia,Serving #Ethiopia🇪🇹 is a great Honor!!!\r\nPer...
4,1.166362e+18,acidrefluxfacts,2019-08-27 14:51:19+00:00,1387.0,4908.0,13496.0,1.0,NaN,Heartburn and Acid Reflux facts and treatment ...
...,...,...,...,...,...,...,...,...,...
18004,9.746740e+05,ethiopmedia,NaT,NaN,NaN,NaN,NaN,NaN,NaN
18005,3.060690e+05,_kheff,NaT,NaN,NaN,NaN,NaN,NaN,NaN
18006,2.110160e+05,realseyoum,NaT,NaN,NaN,NaN,NaN,NaN,NaN
18007,1.451175e+18,fdreservice,NaT,NaN,NaN,NaN,NaN,NaN,NaN


## Finalizing

In [66]:
replys["referenced_tweets.id"]

0       1477111665963548675
1       1477283004603109376
2       1477372129607917577
3       1478112383004463112
4       1478112383004463112
               ...         
2651    1607689856636813313
2652    1608013830201688064
2653    1608860599483760646
2654    1608834209225003009
2655    1608859818063978496
Name: referenced_tweets.id, Length: 2656, dtype: int64

In [15]:
print("Mentions' or replys' column names")

num = 0
for i in edges_mention.columns:
    print(num, ":", i)
    num += 1

print(len(edges_mention), len(replys))
#print("Mentions/Replys")
    
#num = 0
#for i in edges_mention/replys.columns:
#    print(num, ":", i)
#    num += 1

Mentions' or replys' column names
0 : Unnamed: 0
1 : created_at
2 : id
3 : author_id
4 : conversation_id
5 : reply_settings
6 : text
7 : lang
8 : public_metrics.retweet_count
9 : public_metrics.reply_count
10 : public_metrics.like_count
11 : public_metrics.quote_count
12 : possibly_sensitive
13 : referenced_tweets.id
14 : referenced_tweets.type
15 : user.public_metrics.followers_count
16 : user.public_metrics.following_count
17 : user.public_metrics.tweet_count
18 : user.public_metrics.listed_count
19 : user.url
20 : user.id
21 : user.username
22 : user.profile_image_url
23 : user.created_at
24 : user.location
25 : user.description
26 : day
27 : count
28 : mentions
29 : in_reply_to_user_id
30 : connection


NameError: name 'replys' is not defined

In [89]:
# Source, Target(in_reply_to_user_id in all cases), created_at, id, 
# conversation_id, text, lang, public_metrics, referenced_tweets, user.

r = replys.iloc[:, [20, 29, 1, 2, 4, 6, 7, 8, 9, 10, 11, 30]]

m = edges_mention.iloc[:, [20, 29, 1, 2, 4, 6, 7, 8, 9, 10, 11, 30]]

h = edges_ht.iloc[:, [20, 30, 1, 2, 4, 6, 7, 8, 9, 10, 11, 31]]

edges = pd.concat([r, m, h]).reset_index(drop=True)

edges["in_reply_to_user_id"] = edges["in_reply_to_user_id"].astype(np.int64)

edges["end_y"] = edges["created_at"] + pd.Timedelta(days=365)
edges["end_m"] = edges["created_at"] + pd.Timedelta(days=30)
edges["end_w"] = edges["created_at"] + pd.Timedelta(weeks=1)

##### Only mentions to file

In [18]:
m = edges_mention.iloc[:, [20, 29, 1, 2, 4, 6, 7, 8, 9, 10, 11, 30]]
m = m.rename(columns={"user.id":"Source", "in_reply_to_user_id":"Target", "id":"tweet_id"})
m["end_y"] = m["created_at"] + pd.Timedelta(days=365)
m["end_m"] = m["created_at"] + pd.Timedelta(days=30)
m["end_w"] = m["created_at"] + pd.Timedelta(weeks=1)
m.to_csv("data/gephi files/edges_mention2021.csv", index=False)

##### 

In [75]:
num = 0
for i in nodes.columns:
    print(num, ":", i)
    num += 1

0 : user.id
1 : user.username
2 : user.created_at
3 : user.public_metrics.followers_count
4 : user.public_metrics.following_count
5 : user.public_metrics.tweet_count
6 : user.public_metrics.listed_count
7 : user.location
8 : user.description


In [91]:
# rename to include source and target

edges = edges.rename(columns={"user.id":"Source", "in_reply_to_user_id":"Target", "id":"tweet_id"})

edges.to_csv("data/gephi files/en_edges.csv", index=False)

In [92]:
# rename to include id and label

nodes_all = nodes_all.rename(columns={"user.id":"id", "user.username":"Label"})

nodes_all.to_csv("data/gephi files/en_nodes.csv", index=False)